### TensorFlow and other imports

In [0]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

from matplotlib import pyplot as plt

#### Created a function to read the csv and separate what is feature and what is label

In [0]:
def parse_csv(line):
    example_defaults = [[0.], [0.], [0.], [0]]
    parsed_line = tf.decode_csv(line, example_defaults)
    
    features = tf.reshape(parsed_line[:-1], shape=(3,))
    label = tf.reshape(parsed_line[-1], shape=())
    
    return features, label

In [0]:
train_dataset = tf.data.TextLineDataset("colorDatasetNormalized.csv")
train_dataset = train_dataset.skip(1)
train_dataset = train_dataset.map(parse_csv)
train_dataset = train_dataset.shuffle(buffer_size=1000)
train_dataset = train_dataset.batch(32)

# View a single example entry from a batch
features, label = iter(train_dataset).next()
print("example features:", features[0])
print("example label:", label[0])

### Using Keras to make the Neural Net

In [0]:
model = tf.keras.Sequential([
    # First hidden layer
    # Need to specify the input shape,
    # aka the number of features
    # the nummber of nodes and the activation function
    tf.keras.layers.Dense(10, activation="relu", input_shape=(3,)),
    # The others hidden layers only need 
    # the quantity of nodes and the activation function
    tf.keras.layers.Dense(10, activation="relu"),
    # Same here
    tf.keras.layers.Dense(10, activation="relu"),
    # The last layer only need the number of nodes
    # The number here is the number of possible outputs
    # In this case 9 different color classes
    tf.keras.layers.Dense(9)
])

In [0]:
def loss(model, x, y):
    y_ = model(x)
    return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    
    return tape.gradient(loss_value, model.variables)

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

In [0]:
train_loss_results = []
train_accuracy_results = []

num_epochs = 20

for epoch in range(num_epochs):
    epoch_loss_avg = tfe.metrics.Mean()
    epoch_accuracy = tfe.metrics.Accuracy()
    
    for x, y in train_dataset:
        grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.variables),
                                  global_step=tf.train.get_or_create_global_step())
        
        epoch_loss_avg(loss(model, x, y))
        epoch_accuracy(tf.argmax(model(x), axis=1, output_type=tf.int32), y)
    
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

In [0]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)

plt.show()

In [0]:
class_ids = [
    "red-ish",
    "green-ish",
    "blue-ish",
    "orange-ish",
    "yellow-ish",
    "pink-ish",
    "purple-ish",
    "brown-ish",
    "grey-ish"
]
predict_data = tf.convert_to_tensor([
    [0.949019608, 0.917647059, 0.980392157], # pink-ish
    [0.537254902, 0.996078431, 0.549019608], # green-ish
    [0.466666667, 0.525490196,0.278431373], # I think its blue-ish
])

predictions = model(predict_data)

for i, logits in enumerate(predictions):
    class_idx = tf.argmax(logits).numpy()
    name = class_ids[class_idx]
    print("Example {} prediction: {}".format(i, name))